In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from collections import Counter
import os
from argparse import Namespace

In [0]:
flags = Namespace(
    train_file='bahceli_33File_83K_texts.txt',
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['değerli','türk','milleti'],
    predict_top_k=5,
    
)

In [0]:
def get_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'r', encoding='utf-8') as f:
        text = f.read()
    text=text.lower()
    text = text.split()
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Vocabulary size', n_vocab)

    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text

def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

In [0]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state

    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))

In [0]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer


In [0]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()
    
    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])

    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    #print(' '.join(words))
    return ' '.join(words)

In [0]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
        flags.train_file, flags.batch_size, flags.seq_size)

    net = RNNModule(n_vocab, flags.seq_size,
                    flags.embedding_size, flags.lstm_size)
    net = net.to(device)

    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0

    for e in range(200):
        batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
        state_h, state_c = net.zero_state(flags.batch_size)
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1
            net.train()

            optimizer.zero_grad()

            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            loss_value = loss.item()

            loss.backward()

            state_h = state_h.detach()
            state_c = state_c.detach()

            _ = torch.nn.utils.clip_grad_norm_(
                net.parameters(), flags.gradients_norm)

            optimizer.step()

            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 200),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))

            if iteration % 1000 == 0:
                predict(device, net, flags.initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)
                

In [124]:
"""if __name__ == '__main__':
    main()  """

"if __name__ == '__main__':\n    main()  "

In [125]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
    flags.train_file, flags.batch_size, flags.seq_size)

net = RNNModule(n_vocab, flags.seq_size,
                flags.embedding_size, flags.lstm_size)
net = net.to(device)

criterion, optimizer = get_loss_and_train_op(net, 0.01)

iteration = 0

for e in range(20):
    batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
    state_h, state_c = net.zero_state(flags.batch_size)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for x, y in batches:
        iteration += 1
        net.train()

        optimizer.zero_grad()

        x = torch.tensor(x).to(device)
        y = torch.tensor(y).to(device)

        logits, (state_h, state_c) = net(x, (state_h, state_c))
        loss = criterion(logits.transpose(1, 2), y)

        loss_value = loss.item()

        loss.backward()

        state_h = state_h.detach()
        state_c = state_c.detach()

        _ = torch.nn.utils.clip_grad_norm_(
            net.parameters(), flags.gradients_norm)

        optimizer.step()

        if iteration % 200 == 0:
            print('Epoch: {}/{}'.format(e, 200),
                  'Iteration: {}'.format(iteration),
                  'Loss: {}'.format(loss_value))

net.eval()
words=flags.initial_words  
state_h, state_c = net.zero_state(1)
state_h = state_h.to(device)
state_c = state_c.to(device)
for w in words:
    ix = torch.tensor([[vocab_to_int[w]]]).to(device)
    with torch.no_grad():
      output, (state_h, state_c) = net(ix, (state_h, state_c))

_, top_ix = torch.topk(output[0], k=5)
choices = top_ix.tolist()
choice = np.random.choice(choices[0])

words.append(int_to_vocab[choice])

for _ in range(100):
    ix = torch.tensor([[choice]]).to(device)
    with torch.no_grad():
      output, (state_h, state_c) = net(ix, (state_h, state_c))

    _, top_ix = torch.topk(output[0], k=5)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])
    words.append(int_to_vocab[choice])

a=" ".join(words)
print(a)
          

Vocabulary size 19509
Epoch: 1/200 Iteration: 200 Loss: 6.572632789611816
Epoch: 2/200 Iteration: 400 Loss: 6.024259090423584
Epoch: 3/200 Iteration: 600 Loss: 4.545770645141602
Epoch: 4/200 Iteration: 800 Loss: 3.965629816055298
Epoch: 6/200 Iteration: 1000 Loss: 3.5176002979278564
Epoch: 7/200 Iteration: 1200 Loss: 3.1788203716278076
Epoch: 8/200 Iteration: 1400 Loss: 2.7029926776885986
Epoch: 9/200 Iteration: 1600 Loss: 2.440680980682373
Epoch: 11/200 Iteration: 1800 Loss: 2.334904909133911
Epoch: 12/200 Iteration: 2000 Loss: 2.0833723545074463
Epoch: 13/200 Iteration: 2200 Loss: 1.8802564144134521
Epoch: 14/200 Iteration: 2400 Loss: 1.7639431953430176
Epoch: 16/200 Iteration: 2600 Loss: 1.4631963968276978
Epoch: 17/200 Iteration: 2800 Loss: 1.4414101839065552
Epoch: 18/200 Iteration: 3000 Loss: 1.3930637836456299
Epoch: 19/200 Iteration: 3200 Loss: 1.304664134979248
değerli türk milleti alayına yetecek kesinlikle mutlu adil yönelik hizmet ettikleri cezaları için gerekirse netenyahu

In [126]:
a

'değerli türk milleti alayına yetecek kesinlikle mutlu adil yönelik hizmet ettikleri cezaları için gerekirse netenyahu ’ den her türlü tasarım içine övgü . bir evladı olan tüm güç imkânlarıyla ansızın geleceğe az , medyamızın her rüzgâra her inşa ettiği bir güvenli , kompleks seviyesi ağır bir üslubun genelleşmesi türkiye düşmanlarının kadrosuna devlet kasten devlet demektir . bu duygu ve suriye ’ den yavaş çekileceğini etmişlerdir bir süre türklüğün ve kardeşlik duygusuna ilkelerimiz bizatihi 15 kişi türkiye tarafından esad ve kahredici bir araya gelmenin ve bu vesileyle kutluyorum ? suriye arap özel partiler demokratik yere çırpınmasın çok hazırlığını şimdiden türkiye cumhuriyeti büyükelçisi kalacaktır , bu'

değerli türk milleti alayına yetecek kesinlikle mutlu adil yönelik hizmet ettikleri cezaları için gerekirse netenyahu ’ den her türlü tasarım içine övgü . bir evladı olan tüm güç imkânlarıyla ansızın geleceğe az , medyamızın her rüzgâra her inşa ettiği bir güvenli , kompleks seviyesi ağır bir üslubun genelleşmesi türkiye düşmanlarının kadrosuna devlet kasten devlet demektir . bu duygu ve suriye ’ den yavaş çekileceğini etmişlerdir bir süre türklüğün ve kardeşlik duygusuna ilkelerimiz bizatihi 15 kişi türkiye tarafından esad ve kahredici bir araya gelmenin ve bu vesileyle kutluyorum ? suriye arap özel partiler demokratik yere çırpınmasın çok hazırlığını şimdiden türkiye cumhuriyeti büyükelçisi kalacaktır , bu